In [53]:
url1 = (r'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.json')

In [54]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import requests, pandas as pd

In [62]:


data=pd.read_json(url1)
data1 = pd.DataFrame(data).T
data1


,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
AFG,Asia,Afghanistan,2021-11-27,157190,19,46.571,7308,1,-7.857,3945.99,...,None,None,37.746,0.5,64.83,0.511,None,None,None,None
OWID_AFR,None,Africa,2021-11-27,8.63232e+06,8236,4794.71,222501,133,144.857,6284.97,...,None,None,None,None,None,None,None,None,None,None
ALB,Europe,Albania,2021-11-27,199137,405,449.857,3085,8,7.143,69314.9,...,7.1,51.2,None,2.89,78.57,0.795,None,None,None,None
DZA,Africa,Algeria,2021-11-27,209980,163,163,6052,6,5,4706.32,...,0.7,30.4,83.741,1.9,76.88,0.748,None,None,None,None
AND,Europe,Andorra,2021-11-27,16712,0,89.429,131,0,0.143,216046,...,29,37.8,None,None,83.73,0.868,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLF,Oceania,Wallis and Futuna,2021-11-22,None,None,None,None,None,None,None,...,None,None,None,None,79.94,None,None,None,None,None
OWID_WRL,None,World,2021-11-27,2.61075e+08,416278,555395,5.19514e+06,5736,6953,33152.5,...,6.434,34.635,60.13,2.705,72.58,0.737,None,None,None,None
YEM,Asia,Yemen,2021-11-27,9981,2,3.714,1945,2,1,327.346,...,7.6,29.2,49.542,0.7,66.12,0.47,None,None,None,None
ZMB,Africa,Zambia,2021-11-27,210138,20,11.571,3667,0,0,11106.3,...,3.1,24.7,13.938,2,63.89,0.584,None,None,None,None
